In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
%matplotlib inline
from keras.layers import Dense, Convolution2D, UpSampling2D, MaxPooling2D, ZeroPadding2D, Flatten, Dropout, Reshape
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
print(os.listdir("../input"))

In [7]:
pwd

In [12]:
#chcemy się znaleźć w /kaggle/input
%cd input

In [13]:
all_data = pd.read_csv('fer2013/fer2013.csv')

In [14]:
all_data.head()

In [15]:
data = all_data.values

In [16]:
y = data[:, 0]
pixels = data[:, 1]

In [17]:
X = np.zeros((pixels.shape[0], 48*48))
for idx_x in range(X.shape[0]):
    p = pixels[idx_x].split(' ')
    for idx_y in range(X.shape[1]):
        X[idx_x, idx_y] = int(p[idx_y])

In [18]:
X

In [19]:
y

In [20]:
all_data = X

In [21]:
for idx_x in range(42, 45):
    plt.figure(idx_x)
    plt.imshow(all_data[idx_x].reshape((48, 48)), interpolation='none', cmap='gray')
plt.show()

In [22]:
all_data = all_data / 255

In [23]:
all_data

In [25]:
X_train = all_data[0:28710, :]
Y_train = y[0:28710]
X_val = all_data[28710:32300, :]
Y_val = y[28710:32300]

print (X_train.shape, Y_train.shape)
print (X_val.shape, Y_val.shape)

In [26]:
X_train = X_train.reshape((X_train.shape[0], 1 , 48, 48 ))
X_val = X_val.reshape((X_val.shape[0], 1 ,48, 48))

In [27]:
X_train.shape

In [28]:
y_ = np_utils.to_categorical(y, 7)

print (y.shape)
print (y_.shape)

In [29]:
Y_train = y_[:28710]
Y_val = y_[28710:32300]

print (X_val.shape, Y_val.shape)

In [30]:
datagenerator = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.0,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False, 
        vertical_flip=False)  

datagenerator.fit(X_train)

In [31]:
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

In [32]:
def swish_activation(x):
    return (K.sigmoid(x) * x)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(1,48,48)))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation=swish_activation))
model.add(Dropout(0.4))
model.add(Dense(7 , activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam' ,
              metrics=['accuracy'])

print(model.summary())

In [33]:
batch_size = 128
epochs = 14

model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])
steps_per_epoch = len(X) // batch_size
validation_steps = len((X_val, Y_val)) // batch_size

In [ ]:
history = model.fit_generator(datagenerator.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X.shape[0] // batch_size,
                    callbacks=[lr_reduce],
                    validation_data=(X_val, Y_val),
                    epochs = epochs, verbose = 2)

In [ ]:
weights = model.get_weights()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from IPython.display import FileLink 
FileLink(r'model.h5')

In [ ]:
model.save("/kaggle/working/model.h5")

In [ ]:
model

In [ ]:
with open("/kaggle/working/test", "w"):
    pass